<a href="https://colab.research.google.com/github/saurabhIU/Deep-Learning/blob/master/Speech_Denoising_Using_1D_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Data Preparation

In [1]:

import librosa
import librosa.display as disp
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import time;
print(tf.__version__)

1.13.1


## Load Data (Training Clean and Training Noisy data)

In [0]:
s_clean, sr_clean=librosa.load('train_clean_male.wav',sr=None)
S_clean=librosa.stft(s_clean, n_fft=1024, hop_length=512)
Y_train = np.abs(S_clean).T

s_dirty, sr_noisy=librosa.load('train_dirty_male.wav',sr=None)
S_dirty=librosa.stft(s_dirty, n_fft=1024, hop_length=512)

X_train = np.abs(S_dirty).T


In [3]:
print(f'Shape of clean signal is {Y_train.shape} and shape of noisy signal is {X_train.shape}')

Shape of clean signal is (2459, 513) and shape of noisy signal is (2459, 513)


## Helper Functions

In [0]:
def initialise_weights(shape):
    return tf.Variable(tf.truncated_normal(shape, stddev=0.05))
  
def get_input_batch(batch_size, data, labels):
    '''
    Return a total of `batch_size` random samples and labels. 
    '''
    index = np.random.choice(data.shape[0], batch_size)
    x = [data[i] for i in (index)]
    y = [labels[i] for i in (index)]
    return np.asarray(x), np.asarray(y)
  
def flatten_layer(layer):
    # Get the shape of the input layer.
    layer_shape = layer.get_shape()

    # The shape of the input layer is assumed to be:
    # layer_shape == [num_images, img_height, img_width, num_channels]

    # The number of features is: img_height * img_width * num_channels
    # We can use a function from TensorFlow to calculate this.
    num_features = layer_shape[1:4].num_elements()
    
    # Reshape the layer to [num_images, num_features].
    # Note that we just set the size of the second dimension
    # to num_features and the size of the first dimension to -1
    # which means the size in that dimension is calculated
    # so the total size of the tensor is unchanged from the reshaping.
    layer_flat = tf.reshape(layer, [-1, num_features])

    # The shape of the flattened layer is now:
    # [num_images, img_height * img_width * num_channels]

    # Return both the flattened layer and the number of features.
    return layer_flat, num_features

## Neural Network Configuration

In [5]:
EPOCHS = 400
Batch_Size = 100
kernel_size = 5

kernel_num1 = 16
kernel_num2 = 32

fully_connected_size = 513

X = tf.placeholder("float", [Batch_Size,1,513,1])
Y = tf.placeholder("float", [Batch_Size,1,513,1])

filters = {
              'wl1': tf.get_variable('W1', shape=(1,kernel_size,1,kernel_num1), initializer=tf.initializers.he_normal()),
              'wl2': tf.get_variable('W2', shape=(1,kernel_size,kernel_num1,kernel_num2), initializer=tf.initializers.he_normal()),
              'wfc': tf.get_variable('W3', shape=(None,513), initializer=tf.initializers.he_normal()),
    
          }

biases = {
              'bl1': tf.get_variable('B1', shape=(kernel_num1), initializer=tf.initializers.he_normal()),
              'bl2': tf.get_variable('B2', shape=(kernel_num2), initializer=tf.initializers.he_normal()),
              'bl3': tf.get_variable('B3', shape=(fully_connected_size), initializer=tf.initializers.he_normal()),
    
          }

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
#Convolution Layer 
# kernel tensor of shape [filter_width, in_channels, out_channels]

def build_convolutional_layer(input,filter_num,bias_num,kernel_num):
    
    
    conv_layer = tf.nn.conv2d(input,filters[filter_num],strides=[1, 1, 1, 1],padding='SAME')
    
    conv_layer += biases[bias_num]
    
    conv_layer = tf.nn.max_pool (value=conv_layer,
                                  ksize=[1, 2, 2, 1],
                                  strides=[1, 2, 2, 1],
                                  padding='SAME')
    
    conv_layer = tf.nn.relu(conv_layer)
    
    return conv_layer
      

In [0]:
def conv_nn(input):
  
  layer1 = build_convolutional_layer(input,'wl1','bl1',kernel_num1)
  print(layer1)
  
  layer2 = build_convolutional_layer(layer1,'wl2','bl2',kernel_num2)
  print(layer2)
  
  # Fully connected layer
  layer_flat, num_features  = flatten_layer(layer2)

  output = tf.matmul(layer_flat,filters['W3']) + biases['bl3']
  
  output = tf.nn.relu(output)
  
  return output

In [8]:
logits = conv_nn(X)
cost = tf.losses.mean_squared_error(Y,logits)
optimizer = tf.train.AdamOptimizer().minimize(cost)

Tensor("Relu:0", shape=(100, 1, 257, 16), dtype=float32)
Tensor("Relu_1:0", shape=(100, 1, 129, 32), dtype=float32)
Tensor("Reshape:0", shape=(100, 4128), dtype=float32)
4128


ValueError: ignored

## Train Convolutional Neural Network

In [0]:
sess =  tf.Session() 
    
sess.run(tf.global_variables_initializer())
  
tic = time.time()
for i in range(EPOCHS):
    x_batch, y_batch = get_input_batch(Batch_Size,X_train,Y_train)
    c,_ = sess.run([cost, optimizer],feed_dict={X: x_batch, Y: y_batch})
    if i % 100 == 0:
      print(f'Epoch: {i},training loss:{c}')
toc = time.time()
print(f'Time taken for training is {toc-tic}')

## Denoise train noisy signal by feeding it through trained network

In [0]:
prediction = sess.run(logits,feed_dict={X: X_train, Y: X_train})

## Recover speech spectrogram

In [0]:

prediction_complex = np.multiply(np.divide(X_train,X_absolute),prediction)

## Recover Time domain signal

In [0]:
prediction_timedomain = librosa.istft(prediction_complex,hop_length=512, win_length=1024)